In [1]:
import cdsapi
import geopandas as gpd
from time import sleep

---

In [2]:
client = cdsapi.Client()

2025-02-10 15:14:00,375 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-10 15:14:00,376 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


In [3]:
path_to_shapefiles = "/Users/cooper/Desktop/CAMELS-CH/data/catchment_delineations/CAMELS_CH_catchments.shp"

gdf = gpd.read_file(path_to_shapefiles).to_crs(epsg=4326)

In [4]:
def get_bounding_box(gdf):
    """
    Get the bounding box coordinates for a GeoDataFrame.
    
    Args:
        gdf (gpd.GeoDataFrame): GeoDataFrame containing geometries
        
    Returns:
        tuple: Bounding box coordinates in format (North, West, South, East)
        
    Example:
        >>> bbox = get_bounding_box(gdf)
        >>> print(bbox)
        (47.808, 5.956, 45.817, 10.492)
    """
    total_bounds = gdf.total_bounds
    # total_bounds returns (minx, miny, maxx, maxy)
    # Convert to (North, West, South, East) format
    west = total_bounds[0]
    south = total_bounds[1]
    east = total_bounds[2]
    north = total_bounds[3]
    
    return (north, west, south, east)

bbox = get_bounding_box(gdf)
bbox

(np.float64(48.02963712004707),
 np.float64(5.860068506597615),
 np.float64(45.68298143391117),
 np.float64(10.596744444353288))

In [5]:
north, west, south, east = bbox

for year in range(1982, 1992):
    temp_request = {
        "format": "netcdf",
        "download_format": "zip",
        "variable": "2m_temperature",
        "year": [str(year)],
        "month": [f"{i:02d}" for i in range(1, 13)],
        "day": [f"{i:02d}" for i in range(1, 32)],
        "time": ["00:00", "06:00", "12:00", "18:00"],
        "area": [north, west, south, east],
    }

    precip_request = {
        "format": "netcdf",
        "download_format": "zip",
        "variable": "total_precipitation",
        "year": [str(year)],
        "month": [f"{i:02d}" for i in range(1, 13)],
        "day": [f"{i:02d}" for i in range(1, 32)],
        "time": ["00:00"],
        "area": [north, west, south, east],
    }

    client.retrieve(
        "reanalysis-era5-land",
        temp_request,
        f"/Users/cooper/Desktop/CAMELS-CH/data/ERA5_land_1981-2020/2m_temperature_{year}.nc",
    )

    client.retrieve(
        "reanalysis-era5-land",
        precip_request,
        f"/Users/cooper/Desktop/CAMELS-CH/data/ERA5_land_1981-2020/total_precipitation_{year}.nc",
    )

    sleep(5)

2025-02-10 15:14:02,119 INFO Request ID is 93d86fa8-3d2b-45b1-97a8-8ce330fbf17b
2025-02-10 15:14:02,188 INFO status has been updated to accepted
2025-02-10 15:14:10,652 INFO status has been updated to running
2025-02-10 15:22:21,110 INFO status has been updated to successful
2025-02-10 15:22:22,113 INFO Request ID is 11aebc4f-04a6-4737-8e65-eb1a323eb063           
2025-02-10 15:22:22,223 INFO status has been updated to accepted
2025-02-10 15:22:35,679 INFO status has been updated to running
2025-02-10 15:24:16,066 INFO status has been updated to successful
2025-02-10 15:24:21,870 INFO Request ID is 928bf6be-8730-4212-ba84-d48af99b0601           
2025-02-10 15:24:21,920 INFO status has been updated to accepted
2025-02-10 15:24:30,553 INFO status has been updated to running
2025-02-10 15:30:46,344 INFO status has been updated to successful
2025-02-10 15:30:48,565 INFO Request ID is c2b471d3-75fe-4f44-81fc-3f3490d52282           
2025-02-10 15:30:50,613 INFO status has been updated to acc

d37e4f69b1fee10dfc73f05312414d3e.zip:   0%|          | 0.00/3.19M [00:00<?, ?B/s]

2025-02-10 16:18:38,528 INFO Request ID is 266fb58a-d78e-48a2-8bf3-4f523f79c928
2025-02-10 16:18:38,581 INFO status has been updated to accepted
2025-02-10 16:18:52,034 INFO status has been updated to running
2025-02-10 16:20:32,423 INFO status has been updated to successful


ea8880351030642de76d77145a9362a1.zip:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

2025-02-10 16:20:38,078 INFO Request ID is 135b1815-2216-4985-87c3-f070f109e75f
2025-02-10 16:20:38,132 INFO status has been updated to accepted
2025-02-10 16:20:51,523 INFO status has been updated to running
2025-02-10 16:20:59,149 INFO status has been updated to accepted
2025-02-10 16:21:10,819 INFO status has been updated to running
2025-02-10 16:26:56,846 INFO status has been updated to successful


...

e4f186e23cbe0b5f1321d5c00da57176.zip:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

2025-02-10 16:38:16,689 INFO Request ID is bf769563-83d1-4d6a-b6b3-5a82af470596
2025-02-10 16:38:16,972 INFO status has been updated to accepted
2025-02-10 16:38:25,273 INFO status has been updated to running
2025-02-10 16:44:40,391 INFO status has been updated to successful


2d23fee2bafb5602e9d3d2af74c9e5a.zip:   0%|          | 0.00/3.18M [00:00<?, ?B/s]

2025-02-10 16:44:41,137 INFO Request ID is 1041f43b-6381-45d4-b192-0b5b35960aa0
2025-02-10 16:44:41,197 INFO status has been updated to accepted
2025-02-10 16:44:49,999 INFO status has been updated to running
2025-02-10 16:45:14,206 INFO status has been updated to accepted
2025-02-10 16:45:31,354 INFO status has been updated to running
2025-02-10 16:46:35,585 INFO status has been updated to successful


b84ed3b6be70926b220ca8b99cf29900.zip:   0%|          | 0.00/1.05M [00:00<?, ?B/s]